# From image idea to Kibana dashboard using AI

This notebook is based on the article [From image idea to Kibana dashboard using AI](https://www.elastic.co/search-labs/blog/from-image-idea-to-kibana-dashboard-using-ai). With the following code, we can generate a Kibana dashboard from an image.

## Install dependencies

In [ ]:
%pip install elasticsearch pydantic langchain langchain-openai -q


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [40]:
import requests
import base64
import json
import os
import uuid
from getpass import getpass
from typing import Any, Dict, List, Literal, Optional

from elasticsearch import Elasticsearch
from langchain.chat_models import init_chat_model
from pydantic import BaseModel, Field

In [25]:
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")
os.environ["ELASTICSEARCH_API_KEY"] = getpass("Enter your Elasticsearch API key: ")
os.environ["ELASTICSEARCH_URL"] = getpass("Enter your Elasticsearch URL: ")
os.environ["KIBANA_URL"] = getpass("Enter your Kibana URL: ")

## Defining the dashboard schema

In [ ]:
class Visualization(BaseModel):
    title: str = Field(description="The dashboard title")
    type: List[Literal["pie", "bar", "metric"]]
    field: Optional[str] = Field(
        description="The field that this visualization use based on the provided mappings"
    )


class Dashboard(BaseModel):
    title: str = Field(description="The dashboard title")
    visualizations: List[Visualization]

## Loading the json templates

There are 3 templates for each visualization type:
- pie
- bar
- metric

The templates are in the templates folder.

The templates are in the following format:
- insBar.json
- insPie.json
- insMetric.json

You can find the templates here: https://github.com/Delacrobix/mcp_dashboards/tree/notebook/templates download it and store it in the *templates* folder.


In [41]:
templates = {}
base_dir = os.getcwd()
template_dir = os.path.join(base_dir, "templates")

for vis_type in ["pie", "bar", "metric"]:
    template_file = os.path.join(template_dir, f"lns{vis_type.capitalize()}.json")

    try:
        with open(template_file, "r") as f:
            templates[vis_type] = json.load(f)
    except FileNotFoundError:
        print(f"Warning: Template file {template_file} not found")
        templates[vis_type] = {}

    if not templates:
        print("No templates found")
        break

    print(f"Loaded {len(templates)} templates")

Loaded 1 templates
Loaded 2 templates
Loaded 3 templates


Function to insert the values generated by the LLM into the template:

In [28]:
def fill_template_with_analysis(
    template: Dict[str, Any],
    visualization: Visualization,
):
    template_str = json.dumps(template)
    replacements = {
        "{title}": visualization.title,
    }
    if visualization.field:
        replacements["{field}"] = visualization.field
    for placeholder, value in replacements.items():
        template_str = template_str.replace(placeholder, str(value))

    return json.loads(template_str)

Retrieve index mappings for the index that the dashboard is based on.

In [29]:
INDEX_NAME = "kibana_sample_data_logs"

es_client = Elasticsearch(
    [os.getenv("ELASTICSEARCH_URL")],
    api_key=os.getenv("ELASTICSEARCH_API_KEY"),
)

result = es_client.indices.get_mapping(index=INDEX_NAME)
index_mappings = result[list(result.keys())[0]]["mappings"]["properties"]

## Loading image 
You can download and use testing images here: https://github.com/Delacrobix/mcp_dashboards/tree/notebook/imgs

In [30]:
IMAGE_PATH = "dashboard.png"

image_base64 = base64.b64encode(open(IMAGE_PATH, "rb").read()).decode("utf-8")

In [ ]:
prompt = f"""
    You are an expert in analyzing Kibana dashboards from images for the version 9.0.0 of Kibana.

    You will be given a dashboard image and a Elasticsearch index mappings.

    Below is the index mappings for the index that the dashboard is based on.
    Use this to help you understand the data and the fields that are available.

    Index Mappings:
    {index_mappings}
    
    There are some things to consider:
    - field: The field that is relevant for the visualization, this is the data that will be used to generate the chart and can be found in the index mappings.
    - If you choose a field that has multi-field, you can use the field name with the suffix .keyword to get the multi-field. For example: 
        'field_name': {{
            'type': 'text',
            'fields': {{
                'keyword': {{
                    'type': 'keyword',
                }}
            }}
        }}
        Here you can use the field_name.keyword to get the multi-field.
        
        'field_name': {{
            'type': 'keyword',
        }}
        Here you can use the field_name to get the single field.
    - Only include the fields that are relevant for each visualization, based on what is visible in the image. 
    """

message = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": prompt},
            {
                "type": "image",
                "source_type": "base64",
                "data": image_base64,
                "mime_type": "image/png",
            },
        ],
    }
]


try:
    llm = init_chat_model("gpt-4.1-mini")
    llm = llm.with_structured_output(Dashboard)
    response = llm.invoke(message)

    dashboard_values = response

    print("Dashboard values generated by the LLM successfully")
except Exception as e:
    print(f"Failed to analyze image and match fields: {str(e)}")

Dashboard values:  title='Web Traffic Dashboard' visualizations=[Visualization(title='Visits', type=['metric'], field=None), Visualization(title='Most used OS', type=['pie'], field='machine.os.keyword'), Visualization(title='Unique Visitors', type=['metric'], field=None), Visualization(title='State Geo Dest', type=['bar'], field='geo.dest')]


Filling the template with the values generated by the LLM:

In [37]:
panels = []
for vis in dashboard_values.visualizations:
    for vis_type in vis.type:
        template = templates.get(vis_type, templates.get("bar", {}))
        filled_panel = fill_template_with_analysis(template, vis)
        panels.append(filled_panel)

## Generate the dashboard

Here is called the API /api/generate-dashboard. The templates with the values generated by the LLM are sent to the API.

In [38]:
try:
    dashboard_id = str(uuid.uuid4())

    # post request to create the dashboard endpoint
    url = f"{os.getenv('KIBANA_URL')}/api/dashboards/dashboard/{dashboard_id}"

    dashboard_config = {
        "attributes": {
            "title": "Generated Dashboard by LLM",
            "description": "Generated by AI",
            "timeRestore": False,
            "panels": panels,  # Visualizations with the values generated by the LLM
            "timeFrom": "now-7d/d",
            "timeTo": "now",
        },
    }

    headers = {
        "Content-Type": "application/json",
        "kbn-xsrf": "true",
        "Authorization": f"ApiKey {os.getenv('ELASTICSEARCH_API_KEY')}",
    }

    requests.post(
        url,
        headers=headers,
        json=dashboard_config,
    )

    # Url to the generated dashboard
    dashboard_url = f"{os.getenv('KIBANA_URL')}/app/dashboards#/view/{dashboard_id}"

    print("Dashboard URL: ", dashboard_url)
    print("Dashboard ID: ", dashboard_id)

except Exception as e:
    print(f"Failed to create dashboard: {str(e)}")

Dashboard URL:  https://articles-cluster-9-0.kb.us-central1.gcp.cloud.es.io/app/dashboards#/view/644da5a6-adb7-4494-a2ed-52f64459b469
Dashboard ID:  644da5a6-adb7-4494-a2ed-52f64459b469
